In [1]:
import pandas as pd

# CSV 파일을 데이터프레임으로 불러오기
df = pd.read_csv("ranking_data.csv")

# rank와 playerId 열만 선택하여 새로운 데이터프레임 만들기
selected_columns = ['rank', 'playerId']
selected_df = df[selected_columns]

# 결과 출력
print(selected_df)

   rank                          playerId
0     1  0b125013f2fac3966feb887549805cbd
1     2  7963c1386bdbfd6b670a00719f0a6920
2     3  1d876b79f75432619a5d2f93058ab6d2
3     4  05e0afd5b4664eeedeef33b9ddb83609
4     5  697ba5f3dafefbfb616a06cff14e02f5
5     6  240313ba366545d05955396369fb205e
6     7  fa339da933e2a3299c854a080c0220b9
7     7  26326d53a4dc5403b5b0541d3e935e87
8     7  1d01a2d3c8d9e17757e897591b2cb57d
9    10  46c8b7e5d6cd6e15425917359617a85f


In [2]:
import requests
import json

# API 관련 설정
base_url = "https://api.neople.co.kr/cy/players/{}/matches"
api_key = "BXVbXvZz67zzWlrt9PX0RvYdZZUQAEAh"
game_type_id = "rating"
start_date = "<startDate>"
end_date = "<endDate>"
limit = "100"

# match_id를 저장할 빈 리스트 생성
match_ids = []

# 각 playerId에 대해 fetch_matches 호출하여 match_id 추출
for player_id in selected_df['playerId']:
    next_token = None
    while True:
        # fetch_matches 함수 정의
        def fetch_matches(player_id, game_type_id, start_date, end_date, limit, next_token):
            url = f"{base_url.format(player_id)}?gameTypeId={game_type_id}&limit={limit}&apikey={api_key}"
            if next_token:
                url += f"&next={next_token}"
            response = requests.get(url)
            
            return response.json()
        
        matches_data = fetch_matches(player_id, game_type_id, start_date, end_date, limit, next_token)


        



        # match_id 추출
        for match in matches_data['matches']['rows']:
            match_ids.append(match['matchId'])
        
        # 다음 페이지 확인
        if 'next' in matches_data:
            next_token = matches_data['next']
        else:
            break

# 추출된 match_id를 DataFrame에 저장
match_id_df = pd.DataFrame({'matchId': match_ids})

# 중복된 값 제거
match_id_df_uniq = match_id_df.drop_duplicates()

# 결과 확인
print(match_id_df_uniq)

                                               matchId
0    272fbda2d71b9654fe7bb977eb77901f905c847bfb3eba...
1    af57a2b039112a392656de6f3a4ad6451a6e62642154f9...
2    aa90eab5913504bd3ab07d6cfb1fbe7822f1de00457384...
3    028826905518b2fe890b561e73d3291a66f1daa982cdfd...
4    7d1d3cfcfbaf303350917df7b447e7f4ea157b65ee786a...
..                                                 ...
892  f709463bc8e8d5f11fef8e5d5114eb7b2ace5974e2bfdd...
894  bfa1ed364d718bd7c2762cc516a3a95c4273bcf0d707d3...
895  f28f659eb01f8ed47010774ef6851e4f5fae0269e5c40f...
896  bd63da3c5fe961943b4929c21dd62d6ae8f465f2fb6cbf...
901  e7ea82b388c1b48f22628901c0c8c8b230084859ffe111...

[640 rows x 1 columns]


In [3]:
match_id_df_uniq.to_csv('match_ids.csv', index=False)

In [4]:
# 저장된 match IDs를 읽어옵니다.
match_ids_df = pd.read_csv('match_ids.csv')

# API 키를 설정합니다.
api_key = 'BXVbXvZz67zzWlrt9PX0RvYdZZUQAEAh'

# 각 match ID에 대한 데이터를 저장할 빈 리스트를 생성합니다.
all_match_data = []

# 각 match ID에 대해 API를 호출하여 데이터를 가져옵니다.
for match_id in match_ids_df['matchId']:
    url = f'https://api.neople.co.kr/cy/matches/{match_id}?&apikey={api_key}'
    response = requests.get(url)
    
    if response.status_code == 200:
        match_data = response.json()
        all_match_data.append(match_data)
    else:
        print(f"Failed to retrieve data for match ID: {match_id}")



Failed to retrieve data for match ID: 272fbda2d71b9654fe7bb977eb77901f905c847bfb3eba580c5fe3d46fbe2f00
Failed to retrieve data for match ID: aa90eab5913504bd3ab07d6cfb1fbe7822f1de004573841875f7e995bafc0fae
Failed to retrieve data for match ID: 028826905518b2fe890b561e73d3291a66f1daa982cdfd1a7dbc8ff0a4b360c3
Failed to retrieve data for match ID: 7d1d3cfcfbaf303350917df7b447e7f4ea157b65ee786ac6e3e1c7672ad98caf
Failed to retrieve data for match ID: a118c8be924a9379fb92e9a33b00cbb6ebc93255147065967cc962f61dcde27b


In [5]:
import json
with open('match_data.json', 'w', encoding='utf-8') as f:
    json.dump(all_match_data, f, ensure_ascii=False, indent=4)